# Embedding sample notebook

In [ ]:
from powerbiclient import Report, models
from io import StringIO
from ipywidgets import interact
import requests
import pandas as pd

## Following two scenarios are supported for embedding Power BI content:

### [Embedding for your organization](https://docs.microsoft.com/en-us/power-bi/developer/embedded/embedding#embedding-for-your-organization)
#### Following two methods can be used to authenticate the user and get the access token:

- [Device flow authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_by_device_flow)
    
    Code snippet:
    ``` py
    from powerbiclient.authentication import DeviceCodeLoginAuthentication
    device_auth = DeviceCodeLoginAuthentication()
    token_with_device_code = device_auth.get_access_token()
    ```
- [Interactive authentication](https://msal-python.readthedocs.io/en/latest/#msal.PublicClientApplication.acquire_token_interactive)
    
    Code snippet:
    ``` py
    from powerbiclient.authentication import InteractiveLoginAuthentication
    interactive_auth = InteractiveLoginAuthentication()
    token_with_redirect = interactive_auth.get_access_token()
    
    ```
    
Note: Use the access token acquired above while creating the report instance
    
    

### [Embedding for your customers](https://docs.microsoft.com/en-us/power-bi/developer/embedded/embedding#embedding-for-your-customers)
#### For the purpose of demo, a sample report is being embedded:

In [ ]:
# Embed config for sample report
res = requests.get('https://playgroundbe-bck-1.azurewebsites.net/Reports/SampleReport')

access_token = res.json()['EmbedToken']['Token']
embed_url = res.json()['EmbedUrl']

### Create an instance of Power BI Report

- Below snippets are the values for optional view_mode parameter
``` py 
view_mode=models.EmbedMode.VIEW.value
view_mode=models.EmbedMode.EDIT.value
view_mode=models.EmbedMode.CREATE.value
``` 
Default value for the parameter is 
``` py 
view_mode=models.EmbedMode.VIEW.value 
```

- Below snippets are the values for optional [permissions](https://docs.microsoft.com/en-us/javascript/api/overview/powerbi/configure-report-settings#open-in-edit-mode) parameter for embedding the report
``` py 
permissions=models.Permissions.READ.value
permissions=models.Permissions.READWRITE.value
permissions=models.Permissions.COPY.value
permissions=models.Permissions.CREATE.value
permissions=models.Permissions.ALL.value
``` 
Default value for the parameter is 
``` py
permissions=models.Permissions.READ.value 
```

#### Render report in Create mode
User needs to pass below snippet as a parameter to create a new report in embed mode
``` py
dataset_id='POWER_BI_DATASET_ID'
```
Note: 
- Generate Embed URL:
User can generate create report embed url using [Get Dataset In Group](https://docs.microsoft.com/en-us/rest/api/power-bi/datasets/getdatasetsingroup) Power BI Rest API

- Generate Embed Token:
User can generate embed token using [Generate Token For Create In Group](https://docs.microsoft.com/en-us/rest/api/power-bi/embedtoken/reports_generatetokenforcreateingroup) Power BI REST API

- User needs to pass below snippets as parameters if access_token or embed_url are not added as a parameter in report()
``` py
client_id='AZURE_ACTIVE_DIRECTORY_CLIENT_ID'
group_id='POWER_BI_GROUP_ID'
```
- User needs to provide below scopes with client_id while performing Device flow/Interactive authentication to embed report in create mode
``` py
scopes = ["https://analysis.windows.net/powerbi/api/Dataset.ReadWrite.All"
            ,"https://analysis.windows.net/powerbi/api/Content.Create"
            , "https://analysis.windows.net/powerbi/api/Workspace.ReadWrite.All"]
client_id='Azure_ACTIVE_DIRECTORY_CLIENT_ID'
```
- For providing necessary permissions user needs to pass below snippet as a parameter
``` py
permissions=models.Permissions.CREATE.value
```


In [ ]:
report = Report(access_token=access_token, embed_url=embed_url, token_type=models.TokenType.EMBED.value)

### Power BI event handlers

#### Set report loaded event handler

In [ ]:
def loaded_callback(event_details):
    print('Report is loaded')

report.on('loaded', loaded_callback)

#### Set report rendered event handler

In [ ]:
def rendered_callback(event_details):
    print('Report is rendered')

report.on('rendered', rendered_callback)

#### Set report error event handler

In [ ]:
def error_callback(event_details):
    print('error', event_details)

report.on('error', error_callback)

#### Load the report in the output cell

In [ ]:
# Render report
report

#### Change dimensions of embedded report

In [ ]:
# Change dimensions of embedded report
report.set_dimensions(300, 450)

### Export data from a visual of the embedded report

#### Get the active page

In [ ]:
def get_active_page(self):
    
    # Get list of pages
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        
        # Get active page
        if page['isActive'] == True:
            active_page = page
            break
    return active_page

#### Get the visual whose data is to be exported

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']

# Get list of visuals on active page
visuals = report.visuals_on_page(active_page_name)

# Get visual whose type is 'clusteredColumnChart'
visual = next(filter(lambda visual: visual['type'] == 'clusteredColumnChart', visuals))
visual_name = visual['name']

#### Export summarized data from a visual of the embedded report

In [ ]:
summarized_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 20)

print(summarized_exported_data)

#### Export underlying data from a visual of the embedded report

In [ ]:
underlying_exported_data = report.export_visual_data(active_page_name, visual_name, rows = 5, export_data_type = models.ExportDataType.UNDERLYING.value)

print(underlying_exported_data)

### Convert exported visual data into CSV and load into pandas dataframe

In [ ]:
data = StringIO(summarized_exported_data)

df = pd.read_csv(data, sep=",")
df.head()

### Visualize the data

In [ ]:
df.plot.pie(y="Total Units", labels=df.loc[:,'isVanArsdel'].values.tolist(), figsize=(10,15))

### Filter the report by region

In [ ]:
def filter_report(Region):
    region_filter = {
        '$schema': "http://powerbi.com/product/schema#basic",
        'target': {
            'table': "Geo",
            'column': "Region"
        },
        'operator': "In",
        'values': [Region]
    }
    report.remove_filters()
    report.update_filters([region_filter])

### Filter report using ipywidget

In [ ]:
report

In [ ]:
interact(filter_report, Region=['Central', 'East', 'West']);

### Get report bookmarks

In [ ]:
# Get list of bookmarks
bookmarks_list = report.get_bookmarks()

# Convert the list into a dataframe
bookmarks_dataframe = pd.DataFrame(bookmarks_list)

# Rename columns
bookmarks_dataframe.rename(columns={'name':'Name', 'displayName':'Display Name', 'state':'State'}, inplace=True)
bookmarks_dataframe

### Apply report bookmark

In [ ]:
def apply_bookmark(BookmarkDisplayName):
    
    # Get bookmark name corresponding to the display name provided by the user
    bookmark_name = bookmarks_dataframe[bookmarks_dataframe['Display Name'] == BookmarkDisplayName].Name.values[0]
    report.set_bookmark(bookmark_name)

### Apply bookmark using ipywidget

In [ ]:
report

In [ ]:
interact(apply_bookmark, BookmarkDisplayName=bookmarks_dataframe['Display Name'])